# Periods

This notebook aim to show easily how periods work in openfisca

### Initialize a simulation
We need to initialize a simulation to show how periods work in openfisca.

In [51]:
import openfisca_france
tax_benefit_system = openfisca_france.init_tax_benefit_system()
scenario = tax_benefit_system.new_scenario()
scenario.init_single_entity(
    period = 2015,
    parent1 = dict(
        age = 30,
        salaire_de_base = 50000,
        ),
    enfants = [
        dict(age = 12),
        dict(age = 18),
        ],
    )
simulation = scenario.new_simulation()

#### Monthly formula

A simulation's variable when called is calculated at a specific period by its formula (if it exist).
For exemple variable af is a monthly variable.
If called on an annual basis, an error is displayed : 

In [52]:
simulation.calculate('af', '2015')

AssertionError: Requested period 2015 differs from 2015-01 returned by variable af

But when called on it's period (monthly), the result of the variable's formula will be returned

In [ ]:
simulation.calculate('af', '2015-01') #calculate variable af for January 2015

#### Annual formulas

Other formulas only works on a annual basis, thus a monthly call will not work, e.g irpp (impôt sur le revenu) :

In [53]:
simulation.calculate('irpp', '2015-01')

AssertionError: Requested period 2015-01 differs from 2015 returned by variable irpp

It must be called on an annual basis :

In [54]:
simulation.calculate('irpp', '2015') #calculate variable af for January 2015

array([-2400.52807617], dtype=float32)

#### Default period

When a variable is called without period it is the simulation's default period that is used.

In [55]:
simulation.period

Period((u'year', Instant((2015, 1, 1)), 1))

Here default simulation period is year 2015. Thus :

In [56]:
simulation.calculate('irpp')

array([-2400.52807617], dtype=float32)

Simulation.calculate('irpp') is equivalent to simulation.calculate('irpp', '2015')

## Solutions

Specific period calls insure that no errors are made by the user : if a variable is asked to be computed monthly but the user expect to have an annual result (as for 'irpp').

But there is solutions to get the result of a variable on an other period.

### Calculate_add : smal to larger period

The calculate_add method has the same behavior as calculate, except that it sum all variables given their instant of calculus over a period lenght.

In [35]:
print simulation.calculate_add("af", "2015")

[ 1559.88000488]


This result is equivalent to do sum all monthly calculate over the period.

In [36]:
annual_af = 0 #create annual_af equals to 0
for month in range(1,13): # [1,2,...,11,12]
    annual_af += simulation.calculate("af", '2015-{}'.format(month)) #add recursively af for all month in 2014

In [37]:
print annual_af

[ 1559.88000488]


We thus see that annual_af is equal to simulation.calculate_add('af', "2014").

We can test that :

In [38]:
print simulation.calculate_add('af','2015') == annual_af

[ True]


### Calculate_divide : large to smaller period

In [78]:
simulation.calculate_divide("irpp", "2015-01")

array([-200.04400635], dtype=float32)

In [80]:
simulation.calculate("irpp", "2015")/12

array([-200.04400635], dtype=float32)

Be warned, when a variable is calculated on a monthly basis with calculate_divide (or add), the result is stored in cache.

Now you can do :

In [81]:
simulation.calculate('irpp', '2015-01')

array([-200.04400635], dtype=float32)

### Calculate_add_divide :
Some variables are not constant over the year. For exemple variable 'ars' (allocation de rentrée scolaire), is computed only in september. To smooth it over the year, we must uste calculate_add_divide. It simply sum something over a given period, the divide it over sub-periods of that period.

In [102]:
simulation.calculate('ars', '2014')/12

array([ 65.20347595], dtype=float32)

In [108]:
simulation.calculate_add_divide('ars', "2011-08")  # Que fait add_divide ?


array([ 49.04750443], dtype=float32)

In [98]:
simulation.compute_add_divide??

In [83]:
simulation.calculate_output??

We can also use more exotic period by using the class periods. Actually when we do simulation.calculate('irpp','2015'), the '2015' string is converted into an object period.
A more explicit way to do this calculate would be simulation.calculate('irpp', periods.period('2015')

In [57]:
from openfisca_core import periods
periods.period('2015')

Period((u'year', Instant((2015, 1, 1)), 1))

In [58]:
simulation.calculate('irpp', periods.period('2015'))

array([-2400.52807617], dtype=float32)

We can look at periods docstring to understand how it works :

In [70]:
periods.period?

In [71]:
print periods.period.__doc__[: 280] #periods.period? would also display the help, suppress the brackets to get full doc

Return a new period, aka a triple (unit, start_instant, size).

    >>> period(u'2014')
    Period((u'year', Instant((2014, 1, 1)), 1))
    >>> period(u'2014:2')
    Period((u'year', Instant((2014, 1, 1)), 2))
    >>> period(u'2014-2')
    Period((u'month', Instant((2014, 2, 1)),


period is constituted of a three parameters.

A unit : day, month and year
    
A start instant : the date at which it starts

And a size : the number of unit 

periods.period('day','2014-03-01', 32) would be a period going from the first of March to the first of april

If we want to calculate the 'allocation familiales' from the April to July.

In [77]:
af_march_to_july = simulation.calculate_add('af' ,periods.period('month', '2015-04', 4))
print af_march_to_july

[ 519.96002197]
